1. ResNet의 개념
- 기존의 network 구조) x * weight layer * relu * weight layer
- Skip Connection구조) x * weight layer * relu weight layer + x
    - Network가 깊어질수록 앞 단의 Layer 정보가 뒤의 Layer에서는 희석될 수 밖에 없는 단점을 보완.
    - 이전의 정보를 함께 사용하자는 개념.
    
2. DenseNet의 개념
- 모든 Layer에 Skip Connection을 적용.
- 하이퍼파라미터의 증가 (무거운 모델)
- ResNet보다 성능이 좋으나 학술적으로는 ResNet보다 가치가 낮음.

In [1]:
import torch
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [2]:
from torchvision import transforms, datasets
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data/CIFAR_10/',
                train = True, # 학습용
                download = True,
                transform = transforms.Compose([
                    transforms.RandomHorizontalFlip(), # 50% 확률로 상하 반전.
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5),
                                        (0.5, 0.5, 0.5))])), batch_size = 64, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data/CIFAR_10/',
                train = False,
                transform = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5),
                                        (0.5, 0.5, 0.5))])), batch_size = 64) # test는 shuffle이 필요 없음.

Files already downloaded and verified


In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# BasicBlock: conv, bn을 n번씩 진행한 후 input을 더해주는 shortcut.
# ResNet 클래스 내의 _make_layer 함수에 사용.
class BasicBlock(nn.Module):
    # in_planes: input 채널에 대한 depth.
    # planes: output 채널에 대한 depth.
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        
        # padding을 좌우에 1로 주는 이유) output feature와 input feature가 같은 크기를 가지게 하기 위해.
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        # 위 output channel을 input channel로 받아줌.
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        # shortcut 과정
        self.shortcut = nn.Sequential()
        
        # 아래의 두 경우는 feature의 크기가 다르기 때문에 더해주는 것이 어려움.
        # 따라서 차원을 맞춰주어야 함.
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(planes))
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, num_classes = 10): # 최종 output layer의 노드 수.
        super(ResNet, self).__init__()
        self.in_planes = 16 # 최초 input channel.
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride = 1)
        self.layer2 = self._make_layer(32, 2, stride = 2)
        self.layer3 = self._make_layer(64, 2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
        
    # Basic Block을 몇 번 만들 것인지.
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [7]:
model = ResNet().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

print('DEVICE: ', DEVICE)
print('MODEL: ', model)

DEVICE:  cpu
MODEL:  ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, 

In [9]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{}({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset), 
                100. * batch_idx / len(train_loader), 
                loss.item()))

In [10]:
def evaluate(model, test_loader):
    model,eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
EPOCHS = 3
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n'.format(epoch, test_loss, test_accuracy))

## PyTorch에서 제공해주는 기본 모델

In [ ]:
import torchvision.models as models
#model = models.resnet18().cuda()
model = models.resnet18().cpu()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
EPOCHS = 3
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, accuracy: {:.2f}%\n'.format(epoch, test_loss, test_accuracy))

- resnet18 = models.resnet18()
- alexnet = models.alexnet()
- vgg16 = models.vgg16()
- squeezenet = models.squeezenet1_0()
- densenet = models.densenet161()
- inception = models.inception_v3()
- googlenet = models.googlenet()
- shufflenet = models.shufflenet_v2_x1_0()
- mobilenet = models.mobilenet_v2()
- resnext50_32x4d = models.resnext50_32x4d()
- wide_resnet50_2 = models.wide_resnet50_2()
- mnasnet = models.mnasnet1_0()